# Validaciones con Pydantic

## Creacion de esquemas

Crearemos un esquema de datos que contenga toda la informacion relacionada con una pelicula. 

```py
from pydantic import BaseModel
...
...
class Movie(BaseModel):
    id: int | None = None
    title: str 
    year: int
    rating: float
    category: str
```
Ya tenemos los atributos de la clase Movie, la cual hereda de BaseModel, y contiene todos los atributos que contiene una pelicula. 

### Que significa: id: int | None = None 

*id: int | None = None* es una forma de declarar un parámetro en una función o en una clase:

- id: Esto es el nombre del parámetro. En este caso, se está declarando un parámetro llamado id.

- *int | None*: Esto es una anotación de tipo. En este caso, significa que el parámetro id puede ser de tipo **int** (número entero) o puede ser **None** (es decir, no tener valor). El operador | denota una unión de tipos, lo que indica que el parámetro puede ser de cualquiera de los tipos mencionados.

- *= None*: Esto es un valor por defecto. En este caso, el valor por defecto para id es None. Esto significa que si no se proporciona un valor para id al llamar a la función, se asumirá que su valor es **None**.

Se puede remplazar por:

```py
id: Optional[int] = None
```

Siempre y cuando se importe:

```py
from typing import Optional
```

### Modificando POST

Ojo tiene un bug, que ahora vamos a corregir 🐞

```py
@app.post('/movies', tags=['movies'])
def create_movie(movie: Movie):  👈  
    movies.append(movie)

    return movie
```
- Quiere decir que va requerir una *movie* de tipo *Movie*

#### Corrigiendo el bug.

Al momento de ir al navegador y hacer una peticion, para consultar la pelicula que se acabo de crear:

    http://127.0.0.1:5000/movies/101

Mostraba el siguiente error:

    if item['id'] == id:
    TypeError: 'Movie' object is not subscriptable

Esto es por el tipo de datos como se esta guardando, pues lo esta guardando como tipo *main.Movie*, asi que antes de guardarlo, hay que convertirlo a diccionario.

```py
@app.post('/movies', tags=['movies'])
def create_movie(movie: Movie):    
    movies.append(dict(movie)) 👈

    return movie
```


### Modificando PUT

```py
@app.put('/movies/{id}', tags=['movies'])
def update_movie(id: int, movie: Movie):
    
    for movie in movies:
        if movie['id'] == id:
            
            movie['title'] = movie.title
            movie['year'] = movie.year
            movie['rating'] = movie.rating
            movie['category'] = movie.category
            break
    
    return movie
```

### Verificando Esquema

![](https://i.imgur.com/wS1AJo0.png)

## Validaciones de tipos de datos

